In [2]:
import numpy as np
import pandas as pd
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier,export_graphviz
import sklearn
from sklearn.preprocessing import LabelEncoder,LabelBinarizer
import os
import subprocess
from sklearn import tree
from io import StringIO
from sklearn import metrics
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.model_selection import  cross_val_score
from sklearn.preprocessing import Imputer

In [3]:
fil = pd.read_csv("C:\\Users\\Charu Saxena\\Documents\\ml\\Crime Prediction Data\\communities-crime-clean.csv")
fil['highcrime'] = fil['ViolentCrimesPerPop'].map(lambda x: True if x > 0.1 else False)

# 1. Decision Trees

a.)	Create a new field “highCrime” which is true if the crime rate per capita (ViolentCrimesPerPop) is greater than 0.1, and false otherwise. What are the percentage of positive and negative instances in the dataset?

In [3]:
mylist=fil['highcrime'].value_counts()
true_percent = (mylist[1]/(fil['highcrime'].shape))*100
false_percent =(mylist[0]/(fil['highcrime'].shape))*100
print("true percetage %2f " %true_percent)
print("false percentage %2f "%false_percent)

true percetage 62.719518 
false percentage 37.280482 


b.	 Use DecisionTreeClassifier to learn a decision tree to predict highCrime on the entire dataset (remember to exclude the crime rate feature from the input feature set so you are not cheating). 

In [4]:
fil1= fil.copy()
del fil1['state']
del fil1['communityname']
del fil1['fold']
del fil1['highcrime']
del fil1['ViolentCrimesPerPop']


 i)What are the training accuracy, precision, and recall for this tree? 

In [5]:
clf = tree.DecisionTreeClassifier()
from sklearn.metrics import precision_recall_fscore_support
without_split_clf = clf.fit(fil1,fil['highcrime'])
prediction = without_split_clf.predict(fil1)
print("Accuracy: %2f" %clf.score(fil1,fil['highcrime']))
print(precision_recall_fscore_support(fil['highcrime'],prediction))

Accuracy: 1.000000
(array([ 1.,  1.]), array([ 1.,  1.]), array([ 1.,  1.]), array([ 743, 1250], dtype=int64))


ii)What are the main features used for classification? 

In [6]:
my_features = fil1.columns
#print(my_features)
imp_feature = clf.feature_importances_
import numpy as np
indices = np.argsort(imp_feature)[::-1]
print ('Feature Ranking') 
for i in range(10):
    print ("{} feature {} ({})".format(i+1,my_features[indices[i]],imp_feature[indices[i]]))
    
#index = np.argsort(imp_feature)[::-1]
#print ('Ranking of features as:') 
#for i in range(10):
    #print(my_features(index))
    #print ("{} Feature no.{} ({})".format(i+1,imp_feature[i],imp_feature[index[i]] ))
 

Feature Ranking
1 feature PctKids2Par (0.3586522413947397)
2 feature racePctWhite (0.09024745266383537)
3 feature racePctHisp (0.04706697512501519)
4 feature PctLess9thGrade (0.02251430488597435)
5 feature PctEmplManu (0.022001457210565678)
6 feature PctSameState85 (0.017885354493215964)
7 feature HousVacant (0.01691551564447387)
8 feature PctImmigRec10 (0.014596466282100392)
9 feature PctSpeakEnglOnly (0.014412149607803038)
10 feature PctHousOwnOcc (0.014165626021920983)


Can you explain why they make sense (or not)?
    
    The important features came out to be population of kids with two parents. This feature is given highest weight according to decision tree and this makes sense in the way that, kids with 2 parents are often vulnerable and lack good parenting. This is may be the reason they are more involved in crime to fulfill their need. Other features include racePctWhite : which means poplulation of people who are white
               racePctHisp: percentage of population that is hispanic, etc. 
    As we are using decisonTree, we use a attribute called important_features_ which uses  total reduction of the criterion. 

c.	Now apply cross-validation (cross_val_score) to do 10-fold cross-validation to estimate the out-of-training accuracy of decision tree learning for this task.
        
        i.	What are the 10-fold cross-validation accuracy, precision, and recall?



In [7]:
print("Prec %f" % cross_val_score(clf , fil1, fil['highcrime'], cv=10, scoring='precision' ).mean())
print ("Recall %f" %cross_val_score(clf , fil1, fil['highcrime'], cv=10, scoring='recall' ).mean())
print ("Accuracy %f" %cross_val_score(clf , fil1, fil['highcrime'], cv=10, scoring='accuracy' ).mean())

Prec 0.795669
Recall 0.780800
Accuracy 0.728508


ii.	Why are they different from the results in the previous test?

    
    The results is different from what we got before because earlier we are testing the model on the same data that we are training. Whereas, here we are using 10 folds cross validation, which involves taking 10 random parts of data and testing on it and training on the rest , and then averaging out the accuracies. This is the reason that we got better results here as cross-valdidation results in less overfitting than previous case.  
    

# 2. Linear Classification



 a.	Use GaussianNB to learn a Naive Bayes classifier to predict highCrime.

        i.	What is the 10-fold cross-validation accuracy, precision, and recall for this method?


In [8]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import average_precision_score
clf2= GaussianNB()
print("Prec %f" % cross_val_score(clf2 , fil1, fil['highcrime'], cv=10, scoring='precision' ).mean())
print ("Recall %f" %cross_val_score(clf2 , fil1, fil['highcrime'], cv=10, scoring='recall' ).mean())
print ("Accuracy %f" %cross_val_score(clf2 , fil1, fil['highcrime'], cv=10, scoring='accuracy' ).mean())

Prec 0.911800
Recall 0.692000
Accuracy 0.761608


In [9]:
fil3 =  fil1.copy()
clf2.fit(fil1,fil['highcrime'])
fil3['predict'] = clf2.predict(fil1)


   Printing top 10 features using NBgaussian
   
   What are the 10 most predictive features? This can be measured by the normalized absolute difference of means for the feature between the two classes:
|μ_T-μ_F |/(σ_T+σ_F )

In [10]:
from operator import itemgetter , attrgetter, methodcaller

In [11]:
predict_true_fil = fil3[fil3['predict'] == True]
predict_false_fil = fil3[fil3['predict'] == False]


In [12]:
Nb = []
for i in range(len(fil1.columns)):
    Nb.append((abs(predict_true_fil[[i]].mean() - predict_false_fil[[i]].mean()) / predict_true_fil[[i]].std() + predict_false_fil[[i]].std() ))
sorted(Nb, key = itemgetter(0), reverse= True)

[PctKids2Par    1.881312
 dtype: float64, pctWInvInc    1.841672
 dtype: float64, PctPersOwnOccup    1.769371
 dtype: float64, medFamInc    1.763879
 dtype: float64, PctFam2Par    1.757952
 dtype: float64, medIncome    1.733898
 dtype: float64, FemalePctDiv    1.679538
 dtype: float64, PctYoungKids2Par    1.59169
 dtype: float64, PctHousLess3BR    1.579829
 dtype: float64, TotalPctDiv    1.575829
 dtype: float64, PctHousOwnOcc    1.529413
 dtype: float64, PctTeen2Par    1.5067
 dtype: float64, PctPopUnderPov    1.477284
 dtype: float64, perCapInc    1.463695
 dtype: float64, MalePctDivorce    1.371709
 dtype: float64, pctWPubAsst    1.362389
 dtype: float64, racePctWhite    1.348088
 dtype: float64, PctUnemployed    1.29353
 dtype: float64, PctIlleg    1.292631
 dtype: float64, PctNotHSGrad    1.278326
 dtype: float64, PctOccupMgmtProf    1.270628
 dtype: float64, PctHousNoPhone    1.266779
 dtype: float64, blackPerCap    1.235194
 dtype: float64, HispPerCap    1.226781
 dtype: float64

The larger this different, the more predictive the feature. Why do these make sense (or not)?
    
        As, we are using NbGaussian the most important feature we should consider is: Mean, Standard Deviation. 
    Here we see that the most pedictive feature is assigned on the basis that the highest difference in the mean values is assigned as the most predictive feature. This is beacuse; higher the difference in means - easier it will be to  divide the dataset into puruer sets thus, better will be the classification of the dataset. Thus using the above factor  of  μ_T-μ_F |/(σ_T+σ_F ) makes sense.
    

iii.	How do these results compare with your results from decision trees, above?

                                Decion Tree	        NbGaussian
            Important features	PctKids2Par 0.361	PctKids2Par  1.88
                                racePctWhite 0.09	pctWInvInc    1.84
    precison, recl and accuracy Prec 0.783762       Prec 0.911800
                                Recall 0.777600     Recall 0.692000
                                Accuracy 0.723490   Accuracy 0.761608
    
    observation:
    •	We observe that both desicion tree classifier and NbGaussian gives “PctKids2Par ”:
    population of children  with two parents as the most predictive feature with importance 
    of  0.36 and 1.88 respectively. 
    •	On the other hand we see that accuracy and presicion for NBGaussian is better 
    than decision Tree. This shows that for NbGaussian most of the true 
     predicted are correct than Desicion Tree. 
    •	Here we also see that, the better the accuracy(preference) and  recall are  
        better the classifier, as we need to know that,it is important to know the correct prediction of the
        high crime area than the number of high crime area which are correctly predicted(precision)   
         .Thus making NBgauss as better classification here. 

    

# Use LinearSVC 

use linear SVC to learn a linear Support Vector Machine model to predict highCrime.

In [13]:
from sklearn.svm import LinearSVC
from sklearn.metrics import average_precision_score
clf3 = LinearSVC()

i.	 What is the 10-fold cross-validation accuracy, precision, and recall for this method?

In [14]:
print("Prec %f" % cross_val_score(clf3 , fil1, fil['highcrime'], cv=10, scoring='precision' ).mean())
print ("Recall %f" %cross_val_score(clf3 , fil1, fil['highcrime'], cv=10, scoring='recall' ).mean())
print ("Accuracy %f" %cross_val_score(clf3 , fil1, fil['highcrime'], cv=10, scoring='accuracy' ).mean())

Prec 0.845405
Recall 0.834400
Accuracy 0.796234


ii.	What are the 10 most predictive features? 

In [15]:
clf3.fit(fil1,fil['highcrime'])
total_feature = fil1.columns
important_feature = clf3.coef_[0]
#important_feature
import numpy as np
indices = np.argsort(important_feature)[::-1]
print ('Feature Ranking') 
for i in range(10):
    print ("{} feature no.{} ({})".format(i+1,total_feature[indices[i]],important_feature[indices[i]]))


Feature Ranking
1 feature no.PersPerOccupHous (1.7551259640220094)
2 feature no.RentHighQ (1.066885449020638)
3 feature no.MalePctDivorce (1.0656963286462522)
4 feature no.NumUnderPov (1.0515482545096204)
5 feature no.NumStreet (1.019155006693703)
6 feature no.PctOccupMgmtProf (1.0146713487893195)
7 feature no.population (1.0023010767345288)
8 feature no.agePct12t21 (0.9882726304886383)
9 feature no.HousVacant (0.9365134658733343)
10 feature no.agePct65up (0.8905730027567204)


This can be measured by the absolute feature weights in the model. Why do these make sense (or not)?
 
     According to SVC the most prdictive feature here is: PersPerOccupHous ;ie mean people per household. This make any sense as, number of people at a houshold is a way connected to how it determies the crime rate of the area. beacuse lagere the number of peoopl in family, it is less likely that they are rich enough to feed and get their chilren ad other family memebers what they wants and thus gives rise to crimes to fulfil their basic needs and pace up with the society.
     

iii.	How do these results compare with your results from decision trees, above?

    we see there is a change in results this is beacuse:
    
    
                                Decion Tree	        SVC
            Important features  PctKids2Par 0.361   PersPerOccuphous  1.75
                                racePctWhite 0.09   RentHIghQ    1.06
    precison, recl and accuracy Prec 0.783762       Prec 0.845
                                Recall 0.777600     Recall 0.8344
                                Accuracy 0.723490   Accuracy 0.796
                                
      we see that the accuracy , precison and recall all are higher for the SVC than 
      Decision tree thus it can be considered better than decison tree. Therotically we 
      know that, SVC is better as it has more flexibile towards the penalties like 
      constraints or thresholds and loss functions and are better used to scale larger 
      samples. Also we know the fact that SVM donot rely on entire training data as 
      decison tree does thus giving more overfitted predictions. Hence the values prooves 
      so. 


# 3. Regression:

a.	Use LinearRegression to learn a linear model directly predicting the crime rate per capita (ViolentCrimesPerPop).

In [16]:
from sklearn  import linear_model
regr = linear_model.LinearRegression()
regr.fit(fil1,fil['ViolentCrimesPerPop'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

i.	Using 10-fold cross-validation, what is the estimated mean-squared-error (MSE) of the model?
    
        Here the MSE value refers Here the MSE refers to the mean value of the squared deviations of the predictions from the true values, over an out-of-sample test space, generated by a model estimated over a particular sample space. 

In [44]:
prediction = regr.predict(fil1)
mse_scores = cross_val_score(regr, fil1 ,fil['ViolentCrimesPerPop'], scoring = 'neg_mean_squared_error' , cv=10)
mse_scores.mean()

-0.020093969304445319

hre we are observing that we are getting the negative valuebeacuse the unified scoring API always maximizes the score, so scores which need to be minimized are negated in order for the unified scoring API to work correctly. The score that is returned is therefore negated when it is a score that should be minimized and left positive if it is a score that should be maximized. 

ii.	What is the MSE on the training set (train on all the data then test on it all)?

In [18]:
from sklearn.metrics import mean_squared_error
mean_squared_error(fil['ViolentCrimesPerPop'], prediction)


0.016516774880307179

here we observe that the mse value is very low as we are testing on the same data as tested which prooves the analogy that smaller the means squared error, the closer you are to finding the line of best line. hence justifying the results. 

iii.	What features are most predictive of a high crime rate? A low crime rate?

        Here coefficients of regression models for selecting and interpreting features. This is based on the idea 
        that when all features are on the same scale, the most important features should have the highest 
        coefficients in the model, while features uncorrelated with the output variables should have coefficient
        values close to zero.
        This approach can work well even with simple linear regression models when the data is not very noisy

In [19]:
total_feature = fil1.columns
important_feature = regr.coef_
#important_feature
import numpy as np
indices = np.argsort(important_feature)[::-1]
index = np.argsort(important_feature)
#indices
print ('Feature Ranking') 
for i in range(10):
    print ("{} Top 10 high feature no.{} ({})".format(i+1,total_feature[indices[i]],important_feature[indices[i]]))
print(" ")
for i in range(10):
    print ("{} Top 10 low feature no.{} ({})".format(i+1,total_feature[index[i]],important_feature[index[i]]))


Feature Ranking
1 Top 10 high feature no.PersPerOccupHous (0.6350881164986149)
2 Top 10 high feature no.PctHousOwnOcc (0.5681332098865255)
3 Top 10 high feature no.MalePctDivorce (0.45851704864166676)
4 Top 10 high feature no.PctRecImmig8 (0.43251055765291013)
5 Top 10 high feature no.MedRent (0.3727277977113885)
6 Top 10 high feature no.medFamInc (0.2879788749460697)
7 Top 10 high feature no.PctEmploy (0.248474316084703)
8 Top 10 high feature no.MalePctNevMarr (0.22672791284180477)
9 Top 10 high feature no.PctPersDenseHous (0.2143525641419764)
10 Top 10 high feature no.OwnOccMedVal (0.21287586365359717)
 
1 Top 10 low feature no.PctPersOwnOccup (-0.6756944788028594)
2 Top 10 low feature no.TotalPctDiv (-0.561924314414664)
3 Top 10 low feature no.whitePerCap (-0.3510157744407789)
4 Top 10 low feature no.PctKids2Par (-0.32265127649627473)
5 Top 10 low feature no.OwnOccLowQuart (-0.3081702191931843)
6 Top 10 low feature no.numbUrban (-0.29644254317756596)
7 Top 10 low feature no.PersPerR

# Ridge Regression


b.	Now use Ridge regression to reduce the amount of overfitting, using RidgeCV to pick the best alpha from among (10, 1, 0.1, 0.01, and 0.001).
    
    i.	What is the estimated MSE of the model under 10-fold CV?
        
            Ridge regression can be used for data interpretation due to its stability and the fact that useful features tend to have non-zero coefficients


In [20]:
from sklearn.linear_model import RidgeCV
ridge = linear_model.RidgeCV(alphas = (10, 1, 0.1, 0.01, 0.001))
ridge.fit(fil1, fil['ViolentCrimesPerPop'])
mse = cross_val_score(ridge, fil1, fil['ViolentCrimesPerPop'], scoring = 'neg_mean_squared_error' ,cv=10)
mse.mean()

-0.019795021348168747

ii.	What is the MSE on the training set (train on all the data then test on it all)?

In [21]:
ridge_predict = ridge.predict(fil1)
mean_squared_error(fil['ViolentCrimesPerPop'], ridge_predict)

0.016763529155169436

iii.	What is the best alpha?
    

In [22]:
ridge.alpha_

1.0

iv.	What does this say about the amount of overfitting in linear regression for this problem?

        linear Regression -0.020093969304445319
        Ridge Regression  -0.019795021348168747
        Mse is the major factor in regression to determine whether the model is over or under-fitted and here we 
        see a that the mse value for the liner regression is very small thus we can say there exists  very less 
        or near to no overfitting. 
    Although Linear regression,  will always provide results based on some estimates it is modeled on from the 
    training data  coefficients of every variable used.  Thus causes an overfitting problem, as each the 
    declared coefficient estimates will not tell the true patterns and relationship in dataset, but rather just the sample data we have.
    whereas, Ridge can be seeing as, add a 'penalty' term and specify some kind of prior knowledge about how the regression should or should not play out which is determied by alpha value.

    

# c.polynomial features

c.	Now use polynomial features to do quadratic (second-order) polynomial regression.

    i.	What is the estimated MSE of the model under 10-fold CV?


In [24]:
from sklearn.preprocessing import PolynomialFeatures
import numpy as np



featuresReg = fil1[list(fil1)].values
labelsReg = fil['ViolentCrimesPerPop'].values
featuresReg_x = np.array(featuresReg)
labelsReg_y = np.array(labelsReg)


clf4 = PolynomialFeatures(degree=2)
X_ = clf4.fit_transform(featuresReg_x)


clf_poly = linear_model.LinearRegression()
clf_poly.fit(X_, labelsReg_y)
predict_ = clf4.fit_transform(featuresReg_x)
y_pred = clf_poly.predict(predict_)


In [25]:
#mse = cross_val_score(ridge, fil1, fil['ViolentCrimesPerPop'], scoring = 'neg_mean_squared_error' ,cv=10)
#mse.mean()
msr_reg = cross_val_score(clf_poly, featuresReg_x,  labelsReg_y, cv=10, scoring='neg_mean_squared_error')
print(" mse %f" % cross_val_score(clf_poly , fil1, fil['highcrime'] , cv=10, scoring='neg_mean_squared_error' ).mean())

 mean -0.131116


ii.	What is the MSE on the training set (train on all the data then test on it all)?

In [26]:
mean_squared_error(fil['ViolentCrimesPerPop'], y_pred)

1.6958021953832663e-28

iii.	Does this mean the quadratic model is better than the linear model for this problem?

     polynomial regression:  mse -0.131116
         linear regression:  mse -0.020093969304445319
         
     We observe that the mse values for both quadratic and linear regression is somehow similar with a minute difference, thus we can say that both are giving similar results. Thus, we can say that the curve we have drawn i both the cases are linear in nature.
     If the results had ben better for polynomial the reason could be that the data points are more covered for polynomial curve.   
     
     
     
     


# 4.	Dirty Data

Repeat the decision tree learning question for the full (non-clean) data set and present the results.

In [30]:
dirty =  pd.read_csv("C:\\Users\\Charu Saxena\\Documents\\ml\\Crime Prediction Data\\communities-crime-full.csv")
dirty['highcrime'] = dirty['ViolentCrimesPerPop'].map(lambda x: True if x > 0.1 else False)

In [31]:
dirty_1 = dirty.drop(['state','communityname','fold','ViolentCrimesPerPop','county','community','highcrime'], axis=1) 

In [32]:
dirty

,state,county,community,communityname,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,...,PopDens,PctUsePubTrans,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop,ViolentCrimesPerPop,highcrime
0,8,?,?,Lakewoodcity,1,0.19,0.33,0.02,0.90,0.12,...,0.26,0.20,0.06,0.04,0.9,0.5,0.32,0.14,0.20,True
1,53,?,?,Tukwilacity,1,0.00,0.16,0.12,0.74,0.45,...,0.12,0.45,?,?,?,?,0.00,?,0.67,True
2,24,?,?,Aberdeentown,1,0.00,0.42,0.49,0.56,0.17,...,0.21,0.02,?,?,?,?,0.00,?,0.43,True
3,34,5,81440,Willingborotownship,1,0.04,0.77,1.00,0.08,0.12,...,0.39,0.28,?,?,?,?,0.00,?,0.12,True
4,42,95,6096,Bethlehemtownship,1,0.01,0.55,0.02,0.95,0.09,...,0.09,0.02,?,?,?,?,0.00,?,0.03,False
5,6,?,?,SouthPasadenacity,1,0.02,0.28,0.06,0.54,1.00,...,0.58,0.10,?,?,?,?,0.00,?,0.14,True
6,44,7,41500,Lincolntown,1,0.01,0.39,0.00,0.98,0.06,...,0.08,0.06,?,?,?,?,0.00,?,0.03,False
7,6,?,?,Selmacity,1,0.01,0.74,0.03,0.46,0.20,...,0.33,0.00,?,?,?,?,0.00,?,0.55,True
8,21,?,?,Hendersoncity,1,0.03,0.34,0.20,0.84,0.02,...,0.17,0.04,?,?,?,?,0.00,?,0.53,True
9,29,?,?,Claytoncity,1,0.01,0.40,0.06,0.87,0.30,...,0.47,0.11,?,?,?,?,0.00,?,0.15,True


In [33]:
#evrything replaced my mean
for i in dirty_1.columns:
    dirty_1[[i]] = dirty_1[[i]].convert_objects(convert_numeric=True)
    dirty_1[[i]] = dirty_1[[i]].fillna(dirty_1[[i]].mean())

D:\anaconda\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  app.launch_new_instance()


In [34]:
dirty_1

,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,...,PolicAveOTWorked,LandArea,PopDens,PctUsePubTrans,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop
0,0.19,0.33,0.02,0.90,0.12,0.17,0.34,0.47,0.29,0.32,...,0.290000,0.12,0.26,0.20,0.060000,0.040000,0.900000,0.500000,0.32,0.140000
1,0.00,0.16,0.12,0.74,0.45,0.07,0.26,0.59,0.35,0.27,...,0.305987,0.02,0.12,0.45,0.163103,0.076708,0.698589,0.440439,0.00,0.195078
2,0.00,0.42,0.49,0.56,0.17,0.04,0.39,0.47,0.28,0.32,...,0.305987,0.01,0.21,0.02,0.163103,0.076708,0.698589,0.440439,0.00,0.195078
3,0.04,0.77,1.00,0.08,0.12,0.10,0.51,0.50,0.34,0.21,...,0.305987,0.02,0.39,0.28,0.163103,0.076708,0.698589,0.440439,0.00,0.195078
4,0.01,0.55,0.02,0.95,0.09,0.05,0.38,0.38,0.23,0.36,...,0.305987,0.04,0.09,0.02,0.163103,0.076708,0.698589,0.440439,0.00,0.195078
5,0.02,0.28,0.06,0.54,1.00,0.25,0.31,0.48,0.27,0.37,...,0.305987,0.01,0.58,0.10,0.163103,0.076708,0.698589,0.440439,0.00,0.195078
6,0.01,0.39,0.00,0.98,0.06,0.02,0.30,0.37,0.23,0.60,...,0.305987,0.05,0.08,0.06,0.163103,0.076708,0.698589,0.440439,0.00,0.195078
7,0.01,0.74,0.03,0.46,0.20,1.00,0.52,0.55,0.36,0.35,...,0.305987,0.01,0.33,0.00,0.163103,0.076708,0.698589,0.440439,0.00,0.195078
8,0.03,0.34,0.20,0.84,0.02,0.00,0.38,0.45,0.28,0.48,...,0.305987,0.04,0.17,0.04,0.163103,0.076708,0.698589,0.440439,0.00,0.195078
9,0.01,0.40,0.06,0.87,0.30,0.03,0.90,0.82,0.80,0.39,...,0.305987,0.00,0.47,0.11,0.163103,0.076708,0.698589,0.440439,0.00,0.195078


In [35]:


print("Prec %f" % cross_val_score(clf , dirty_1, dirty['highcrime'], cv=10, scoring='precision' ).mean())
print ("Recall %f" %cross_val_score(clf , dirty_1, dirty['highcrime'], cv=10, scoring='recall' ).mean())
print ("Accuracy %f" %cross_val_score(clf , dirty_1, dirty['highcrime'], cv=10, scoring='accuracy' ).mean())

clf.fit(dirty_1,dirty['highcrime'])
predict = clf.predict(dirty_1)
print("Accuracy: %2f" %clf.score(dirty_1,dirty['highcrime']))
print(precision_recall_fscore_support(dirty['highcrime'],predict))



important_feature = clf.feature_importances_
#important_feature
import numpy as np
indices = np.argsort(important_feature)[::-1]
index = np.argsort(important_feature)
#indices
print ('Feature Ranking') 
for i in range(10):
    print ("{} Top 10 high feature are{} ({})".format(i+1,total_feature[indices[i]],important_feature[indices[i]]))

Prec 0.807637
Recall 0.785727
Accuracy 0.759258
Accuracy: 1.000000
(array([ 1.,  1.]), array([ 1.,  1.]), array([ 1.,  1.]), array([ 743, 1251], dtype=int64))
Feature Ranking
1 Top 10 high feature arePctKids2Par (0.36209377222520184)
2 Top 10 high feature areracePctWhite (0.0864803639160106)
3 Top 10 high feature areracePctHisp (0.04705294890995394)
4 Top 10 high feature arePctLess9thGrade (0.022131879877015988)
5 Top 10 high feature arePctEmplManu (0.018169093556580387)
6 Top 10 high feature areMedRent (0.01558218495438704)
7 Top 10 high feature arePctNotHSGrad (0.015104477930921609)
8 Top 10 high feature arePctImmigRec10 (0.014592116455610307)
9 Top 10 high feature arePctTeen2Par (0.014105274991453772)
10 Top 10 high feature arePctEmploy (0.013338857383340412)



a.	Are the CV results better or worse? What does this say about the effect of missing values?
 
            previous results:   present results(dirty)
            Prec 0.783762       Prec 0.807637 
            Recall 0.777600     Recall 0.785727
            Accuracy 0.723490   Accuracy 0.759258
   
               We observe that the present results with dirty data is much better as compared to the
               previous  results thus the results gets better. 
    •	Here we observe that in the file of communities-full there are many missing values, which we replaced by 
        the mean value of each column because we cannot train classifier on the basis of missing values beacuse
        our data is normal distributed and mean works good for it.
    •	Now after running decision tree classifier, we got the above results as, where we observe that the here
    the results are much better than the previous cleaned-data. One possible reason could be that there are 
    many attribute missing in the firt case which are added in the dirty set. So, we can sa that these new 
    attributs must be of greater importances thus giving better results. We observe that there are total of 129 attributes in the dirty set where as there 105 attributes.
    


# 5.	Teams 

i.	Experiment with two learning methods other than those described above (one can be a non-linear kernel for SVM) for the classification problem, explaining clearly what you did. Show CV results for both the clean and full datasets.

In [54]:
#we will be using first as logistic Regression:
#for clean_data

from sklearn import linear_model
cl = linear_model.LogisticRegression()

cl.fit(fil1, fil['highcrime'])

rs1 = cross_val_score(cl, fil1, fil['highcrime'], cv=10)
print(rs1)
print ("Accuracy %f" %cross_val_score(cl , fil1, fil['highcrime'], cv=10, scoring='accuracy' ).mean())

important_feature = cl.coef_[0]
#important_feature
import numpy as np
indices = np.argsort(important_feature)[::-1]
index = np.argsort(important_feature)
#indices
print ('Feature Ranking') 
for i in range(10):
    print ("{} Top 10 high feature are {} ({})".format(i+1,total_feature[indices[i]],important_feature[indices[i]]))

[ 0.78        0.865       0.885       0.82914573  0.70854271  0.70351759
  0.81407035  0.82914573  0.83417085  0.80904523]
Accuracy 0.805764
Feature Ranking
1 Top 10 high feature are racepctblack (2.405086903563553)
2 Top 10 high feature are MalePctDivorce (1.4050053011837318)
3 Top 10 high feature are racePctHisp (1.3484991089118727)
4 Top 10 high feature are PctPersDenseHous (1.3083876588909886)
5 Top 10 high feature are PctIlleg (1.0894599454668115)
6 Top 10 high feature are TotalPctDiv (1.0887674363880042)
7 Top 10 high feature are HousVacant (1.0720052625321648)
8 Top 10 high feature are RentHighQ (1.003271102956885)
9 Top 10 high feature are OtherPerCap (0.9569952563112218)
10 Top 10 high feature are MedRentPctHousInc (0.8736641467565817)


In [53]:
#for dirty data
cl_dirty = linear_model.LogisticRegression()
cl_dirty.fit(fil1, fil['highcrime'])
rs1_dirty = cross_val_score(cl_dirty, dirty_1, dirty['highcrime'], cv=10)
print(rs1_dirty)
print ("Accuracy %f" %cross_val_score(cl_dirty , dirty_1, dirty['highcrime'], cv=10, scoring='accuracy' ).mean())

important_feature = cl_dirty.coef_[0]
#important_feature
import numpy as np
indices = np.argsort(important_feature)[::-1]
index = np.argsort(important_feature)
#indices
print ('Feature Ranking') 
for i in range(10):
    print ("{} Top 10 high feature are {} ({})".format(i+1,total_feature[indices[i]],important_feature[indices[i]]))

[ 0.84577114  0.825       0.85        0.82914573  0.81909548  0.83417085
  0.86934673  0.83919598  0.80904523  0.83919598]
Accuracy 0.835997
Feature Ranking
1 Top 10 high feature are racepctblack (2.405086903563553)
2 Top 10 high feature are MalePctDivorce (1.4050053011837318)
3 Top 10 high feature are racePctHisp (1.3484991089118727)
4 Top 10 high feature are PctPersDenseHous (1.3083876588909886)
5 Top 10 high feature are PctIlleg (1.0894599454668115)
6 Top 10 high feature are TotalPctDiv (1.0887674363880042)
7 Top 10 high feature are HousVacant (1.0720052625321648)
8 Top 10 high feature are RentHighQ (1.003271102956885)
9 Top 10 high feature are OtherPerCap (0.9569952563112218)
10 Top 10 high feature are MedRentPctHousInc (0.8736641467565817)


In [38]:
# using random Forest for clean data
from sklearn.ensemble import RandomForestClassifier


In [52]:
clf_Random = RandomForestClassifier(n_jobs=2)
clf_Random.fit(fil1, fil['highcrime'])

rs1 = cross_val_score(clf_Random, fil1, fil['highcrime'], cv=10)
print(rs1)
print ("Accuracy %f" %cross_val_score(clf_Random , fil1, fil['highcrime'], cv=10, scoring='accuracy' ).mean())

from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier

# Build a classification task using 3 informative features


important_feature = clf_Random.feature_importances_
import numpy as np
indices = np.argsort(important_feature)[::-1]
index = np.argsort(important_feature)
indices
print ('Feature Ranking') 
for i in range(10):
    print ("{} Top 10 high feature are {} ({})".format(i+1,total_feature[indices[i]],important_feature[indices[i]]))


[ 0.78        0.865       0.795       0.83417085  0.69346734  0.71356784
  0.79396985  0.7839196   0.83417085  0.81407035]
Accuracy 0.792206
Feature Ranking
1 Top 10 high feature are PctKids2Par (0.13297112329865685)
2 Top 10 high feature are TotalPctDiv (0.08807358200712093)
3 Top 10 high feature are racePctWhite (0.061613424605993214)
4 Top 10 high feature are PctYoungKids2Par (0.038224763755632066)
5 Top 10 high feature are PctPopUnderPov (0.033044489091753025)
6 Top 10 high feature are racepctblack (0.03295572784312699)
7 Top 10 high feature are PctPersDenseHous (0.026578470493311745)
8 Top 10 high feature are PctNotSpeakEnglWell (0.01513295329624495)
9 Top 10 high feature are PctTeen2Par (0.01464472294417066)
10 Top 10 high feature are MalePctDivorce (0.014123039602184164)


In [51]:
#using Random Frest for dirty data

clRandom_dirty = RandomForestClassifier(n_jobs=2)
clRandom_dirty.fit(dirty_1, dirty['highcrime'])
rsrandom_dirty = cross_val_score(clRandom_dirty, dirty_1, dirty['highcrime'], cv=10)
print(rsrandom_dirty)
print ("Accuracy %f" %cross_val_score(clRandom_dirty , dirty_1, dirty['highcrime'], cv=10, scoring='accuracy' ).mean())

important_feature = clRandom_dirty.feature_importances_
#important_feature
import numpy as np
indices = np.argsort(important_feature)[::-1]
index = np.argsort(important_feature)
#indices
print ('Feature Ranking') 
for i in range(10):
    print ("{} Top 10 high feature are {} ({})".format(i+1,total_feature[indices[i]],important_feature[indices[i]]))

[ 0.81094527  0.82        0.805       0.7839196   0.78894472  0.81909548
  0.8241206   0.84422111  0.7839196   0.80904523]
Accuracy 0.830502
Feature Ranking
1 Top 10 high feature are NumIlleg (0.06350120767730814)
2 Top 10 high feature are racePctWhite (0.05685007430847561)
3 Top 10 high feature are PctFam2Par (0.054993521616403106)
4 Top 10 high feature are pctWInvInc (0.05481087897669436)
5 Top 10 high feature are PctKids2Par (0.03347876565297825)
6 Top 10 high feature are PctYoungKids2Par (0.03316051790800406)
7 Top 10 high feature are racepctblack (0.028834194597427282)
8 Top 10 high feature are NumUnderPov (0.02823575595881006)
9 Top 10 high feature are TotalPctDiv (0.02461130675736827)
10 Top 10 high feature are PctIlleg (0.02452433200136153)


    *********************Anlysis*********************
                random forest:            logistic regresion:
               
      clean data-Accuracy 0.792201      Accuracy 0.805764

                   PctIlleg             racepctblack 
                   PctKids2Par          MalePctDivorce 
                 
    dirty data-Accuracy 0.821939         Accuracy 0.835997

                PctKids2Par            racepctblack
                PctFam2Par             MalePctDivorce
                
      observations:
         •	Accuracies in both the cases are more for dirty data as compared to the clean data which is explained above in 4. Dirty data(as there exist now more number of attributes which may proove to be more importance and giving their parts as a part in classification). For both logistic regression and random forest.

     •	When we are considering both the clean data and dirty data logistic regression is giving better results with accuracies of 80% and 83% in case of logistic regression.

    •	Logistic regression gives easier probability scores and gives efficient implementation than random forest. Also in case, of random forest the model is generally observed to overfit for some datasets with noisy classification which is not in case of logistic regression thus providing better results. Random forest tends to be biased for the categorical form of datsset with different levels which is the case here. 
    
   

In [49]:
len(dirty)
dirty.columns

Index(['state', 'county', 'community', 'communityname', 'fold', 'population',
       'householdsize', 'racepctblack', 'racePctWhite', 'racePctAsian',
       ...
       'PopDens', 'PctUsePubTrans', 'PolicCars', 'PolicOperBudg',
       'LemasPctPolicOnPatr', 'LemasGangUnitDeploy', 'LemasPctOfficDrugUn',
       'PolicBudgPerPop', 'ViolentCrimesPerPop', 'highcrime'],
      dtype='object', length=129)

In [50]:
fil.columns

Index(['state', 'communityname', 'fold', 'population', 'householdsize',
       'racepctblack', 'racePctWhite', 'racePctAsian', 'racePctHisp',
       'agePct12t21',
       ...
       'PctBornSameState', 'PctSameHouse85', 'PctSameCity85', 'PctSameState85',
       'LandArea', 'PopDens', 'PctUsePubTrans', 'LemasPctOfficDrugUn',
       'ViolentCrimesPerPop', 'highcrime'],
      dtype='object', length=105)